In [1]:
from Original.ori_prep import *
from Original.ori_data import *
from Original.ori_model import *
from Original.ori_multi_training import *
from Original.out_to_in_op import *

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [20]:
import tensorflow as tf
import midi
import numpy as np

In [3]:
piece=loadPieces('data/music/')

Loaded liz_et2
Loaded scn16_3
Loaded haydn_43_1
Loaded mendel_op30_3
Loaded beethoven_les_adieux_1
Loaded br_rhap
Loaded grieg_zwerge
Loaded haydn_7_1
Loaded muss_8
Loaded schu_143_2
Loaded muss_4
Loaded scn68_12
Loaded clementi_opus36_1_1
Loaded waldstein_3
Loaded liz_et_trans5
Loaded ty_maerz
Loaded ty_juli
Loaded rac_op3_2
Loaded chpn-p3
Loaded gra_esp_4
Loaded scn15_5
Loaded beethoven_opus10_2
Loaded grieg_wedding
Loaded mz_330_1
Loaded haydn_35_2
Loaded chpn_op27_1
Loaded schubert_D935_3
Loaded mendel_op19_4
Loaded schub_d960_1
Loaded haydn_9_1
Loaded mz_545_1
Loaded rac_op33_6
Loaded liz_rhap15
Loaded alb_esp2
Loaded grieg_berceuse
Loaded rac_op23_2
Loaded bach_850
Loaded beethoven_opus22_1
Loaded burg_quelle
Loaded debussy_cc_4
Loaded schubert_D850_4
Loaded chpn-p4
Loaded mendel_op30_1
Loaded pathetique_3
Loaded clementi_opus36_6_1
Loaded mendel_op62_3
Loaded schu_143_1
Loaded chpn_op25_e11
Loaded scn15_4
Loaded schub_d960_4
Loaded mz_570_3
Loaded clementi_opus36_5_1
Loaded scn1

In [4]:
sample_in,sample_out=getPieceSegment(piece)

In [36]:
np.asarray(sample_in).shape
sample_in=np.asarray(sample_in).reshape([128,78,80])

In [43]:
sample_in=tf.convert_to_tensor(sample_in,dtype=tf.float64)

In [44]:
sample_in

<tf.Tensor: shape=(128, 78, 80), dtype=float64, numpy=
array([[[ 0.,  1.,  0., ..., -1., -1.,  0.],
        [ 1.,  0.,  1., ..., -1., -1.,  0.],
        [ 2.,  0.,  0., ..., -1., -1.,  0.],
        ...,
        [75.,  0.,  0., ..., -1., -1.,  0.],
        [76.,  0.,  0., ..., -1., -1.,  0.],
        [77.,  0.,  0., ..., -1., -1.,  0.]],

       [[ 0.,  1.,  0., ..., -1., -1.,  0.],
        [ 1.,  0.,  1., ..., -1., -1.,  0.],
        [ 2.,  0.,  0., ..., -1., -1.,  0.],
        ...,
        [75.,  0.,  0., ..., -1., -1.,  0.],
        [76.,  0.,  0., ..., -1., -1.,  0.],
        [77.,  0.,  0., ..., -1., -1.,  0.]],

       [[ 0.,  1.,  0., ..., -1., -1.,  0.],
        [ 1.,  0.,  1., ..., -1., -1.,  0.],
        [ 2.,  0.,  0., ..., -1., -1.,  0.],
        ...,
        [75.,  0.,  0., ..., -1., -1.,  0.],
        [76.,  0.,  0., ..., -1., -1.,  0.],
        [77.,  0.,  0., ..., -1., -1.,  0.]],

       ...,

       [[ 0.,  1.,  0., ...,  1.,  1.,  0.],
        [ 1.,  0.,  1., ...,  1.

In [46]:
x,y,z=tf.keras.layers.LSTM(4, return_sequences=True, stateful=True, return_state=True)(sample_in)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [48]:
print(x.shape)
print(y.shape)
print(z.shape)


(128, 78, 4)
(128, 4)
(128, 4)


In [5]:
piece_batch=getPieceBatch(piece)

In [72]:
batch_sample_in=piece_batch[0]
print(batch_sample_in.shape)
batch_sample_out=piece_batch[1]
print(batch_sample_out.shape)

(10, 128, 78, 80)
(10, 128, 78, 2)


In [63]:
inputs = tf.keras.Input(shape=(128,78,80))

inputs_rotate= tf.keras.backend.permute_dimensions(inputs,(0,2,1,3)) #(78,128,80)

time_lstm1 = tf.keras.layers.LSTM(80,return_sequences=True)
time_lstm2 = tf.keras.layers.LSTM(3,return_sequences=True)

inter1 = tf.keras.layers.TimeDistributed(time_lstm1)(inputs_rotate) #(78,128,80)
inter2 = tf.keras.layers.TimeDistributed(time_lstm2)(inter1) #(78,128,80)

note_lstm1 = tf.keras.layers.LSTM(3,return_sequences=True)
note_lstm2 = tf.keras.layers.LSTM(3,return_sequences=True)

inter2_rotate= tf.keras.backend.permute_dimensions(inter2,(0,2,1,3)) #(128,78,80)

inter3 = tf.keras.layers.TimeDistributed(note_lstm1)(inter2_rotate)
inter4 = tf.keras.layers.TimeDistributed(note_lstm2)(inter3)

outputs = tf.keras.layers.Dense(2)(inter4)

In [64]:
print(inputs.shape)
print(inputs_rotate.shape)
print(inter1.shape)
print(inter2.shape)
print(inter2_rotate.shape)
print(inter3.shape)
print(inter4.shape)
print(outputs.shape)

(None, 128, 78, 80)
(None, 78, 128, 80)
(None, 78, 128, 80)
(None, 78, 128, 3)
(None, 128, 78, 3)
(None, 128, 78, 3)
(None, 128, 78, 3)
(None, 128, 78, 2)


In [80]:
model=tf.keras.Model(inputs=inputs,outputs=outputs)


In [81]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 128, 78, 80)]     0         
_________________________________________________________________
tf_op_layer_Transpose_17 (Te [(None, 78, 128, 80)]     0         
_________________________________________________________________
time_distributed_36 (TimeDis (None, 78, 128, 80)       51520     
_________________________________________________________________
time_distributed_37 (TimeDis (None, 78, 128, 3)        1008      
_________________________________________________________________
tf_op_layer_Transpose_18 (Te [(None, 128, 78, 3)]      0         
_________________________________________________________________
time_distributed_38 (TimeDis (None, 128, 78, 3)        84        
_________________________________________________________________
time_distributed_39 (TimeDis (None, 128, 78, 3)        84  

In [82]:
model.compile(loss='categorical_crossentropy', optimizer='Adam')

In [85]:
def batch_gen():
    while True:
        batch=getPieceBatch(piece)
        yield (batch[0],batch[1])

In [87]:
gen=batch_gen()
gen

<generator object batch_gen at 0x7fb9e6d62cd0>

In [89]:
model.fit_generator(gen,epochs=20,steps_per_epoch=20)

Epoch 1/20
20/20 [==============================] - 39s 2s/step - loss: 0.0310
Epoch 2/20
20/20 [==============================] - 40s 2s/step - loss: 0.0293
Epoch 3/20
20/20 [==============================] - 38s 2s/step - loss: 0.0283
Epoch 4/20
20/20 [==============================] - 38s 2s/step - loss: 0.0281
Epoch 5/20
20/20 [==============================] - 40s 2s/step - loss: 0.0342
Epoch 6/20
20/20 [==============================] - 38s 2s/step - loss: 0.0393
Epoch 7/20
20/20 [==============================] - 38s 2s/step - loss: 0.0333
Epoch 8/20
20/20 [==============================] - 40s 2s/step - loss: 0.0320
Epoch 9/20
20/20 [==============================] - 38s 2s/step - loss: 0.0294
Epoch 10/20
20/20 [==============================] - 38s 2s/step - loss: 0.0277
Epoch 11/20
20/20 [==============================] - 40s 2s/step - loss: 0.0263
Epoch 12/20
20/20 [==============================] - 38s 2s/step - loss: 0.0250
Epoch 13/20
20/20 [==============================

In [78]:
pred=model.predict(np.asarray(sample_in).reshape(1,128,78,80))
pred.shape
pred

array([[[[-1.0550620e-03,  9.8814047e-04],
         [-1.8512595e-03,  7.9484761e-04],
         [-2.1429136e-03,  1.0121426e-03],
         ...,
         [-4.2161275e-02, -1.7985332e-01],
         [-4.2025354e-02, -1.7958283e-01],
         [-4.2874381e-02, -1.7961159e-01]],

        [[-1.1308468e-04,  1.5475387e-03],
         [-1.1729083e-03,  1.5339856e-03],
         [-9.4266311e-04,  2.5368836e-03],
         ...,
         [-7.6267138e-02, -3.0307859e-01],
         [-7.5431153e-02, -3.0151436e-01],
         [-7.6581381e-02, -3.0038917e-01]],

        [[ 2.0089280e-04,  1.2428381e-03],
         [-8.2197785e-04,  5.3085049e-04],
         [ 6.7224598e-04,  1.8437505e-03],
         ...,
         [-9.4751462e-02, -3.7570331e-01],
         [-9.2851274e-02, -3.7334526e-01],
         [-9.3376607e-02, -3.7141582e-01]],

        ...,

        [[ 4.8230803e-03,  4.5317872e-03],
         [ 8.6049177e-03,  7.9765460e-03],
         [ 1.6099118e-02,  1.9037379e-02],
         ...,
         [-1.9562711e

In [ ]:
## time axis only
model=tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(24,  return_sequences=True, stateful=True,
                     batch_input_shape=(10, 128, 78)))


In [ ]:
## note axis only


In [14]:
## biaxial model
import tensorflow as tf



In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(3,  return_sequences=True, stateful=True,
                     batch_input_shape=(10, 128, 78)))


In [ ]:
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(512, return_sequences=False))
model.add(Dense(78))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')